In [1]:
# Componentes básicos da rede
from components import Network, Controller
# Funções auxiliares para executar as aplicações
from components.qkd.app import *

import numpy as np

In [2]:
# Criando a rede
rede = Network()

# Criando e definindo um controlador pra rede
controlador = Controller()
controlador.set_network(rede)

In [3]:
#  ________________________________________________________________
# |Simulação: Várias execuções                                     |
# |          _______________________________________________       |
# |         |Execução: geração e atendimento dos requests   |      |
# |         |                    ___________________________|      |
# |         |                   |Requests: apps QKD         |      |
# |         |___________________|___________________________|      |
# |________________________________________________________________|
#

In [4]:
# Propriedades para simulação
n_simulacoes = 100
n_requests = 2
n_total_execucoes = 0

## Coleta dos resultados

In [5]:
# Taxa de sucesso de bits das chaves
taxas_sucesso_chaves_geral = []
taxas_sucesso_chaves_e91 = []
taxas_sucesso_chaves_bb84 = []
taxas_sucesso_chaves_b92 = []

# Médias de sucesso de bits das chaves
medias_sucesso_chaves_geral = dict()

# Vazão da rede
vazao = []
media_vazao = []

In [6]:
# Executa num_simulacoes vezes simulações de requests
for indice, simulacao in enumerate(range(n_simulacoes)):
    
    taxas_sucesso_chaves_e91 = []
    taxas_sucesso_chaves_bb84 = []
    taxas_sucesso_chaves_b92 = []
    rede.set_lattice_topology(3, 4)
        
    requests = generate_qkd_request(rede, n_requests)
    resultados_simulacao = controlador.send_requests(requests)
    
    for indice_execucao in resultados_simulacao:
        # Percorre cada execução e colhe o sucesso nas chaves
        resultado_individual_simulacao = resultados_simulacao[indice_execucao]
        sucesso_chave = resultado_individual_simulacao['key sucess']
        
        # Colhendo o sucesso nas chaves de cada app (Tamamnho da chave recebida/ Tamanho da chave enviada)
        if resultado_individual_simulacao['app'] == 'BB84':
            taxas_sucesso_chaves_bb84.append(sucesso_chave)
            media_sucesso_chaves_bb84 = sum(taxas_sucesso_chaves_bb84) / len(taxas_sucesso_chaves_bb84)
        
        elif resultado_individual_simulacao['app'] == 'E91':
            taxas_sucesso_chaves_e91.append(sucesso_chave)
            media_sucesso_chaves_e91 = sum(taxas_sucesso_chaves_e91) / len(taxas_sucesso_chaves_e91)
            
        elif resultado_individual_simulacao['app'] == 'B92':
            taxas_sucesso_chaves_b92.append(sucesso_chave)
            media_sucesso_chaves_b92 = sum(taxas_sucesso_chaves_b92) / len(taxas_sucesso_chaves_b92)
      
    
    # Salvando o sucesso nas chaves geral da simulação
    lista_combinada = [taxa for sublist in [taxas_sucesso_chaves_bb84, taxas_sucesso_chaves_e91, taxas_sucesso_chaves_b92] for taxa in sublist]
    #print(lista_combinada)
    taxas_sucesso_chaves_geral.append(sum(lista_combinada) / len(lista_combinada))
    
    # Calculando a vazão
    n_execucoes = len(resultados_simulacao)
    vazao.append(n_requests / n_execucoes)
    

In [7]:
print(np.mean(vazao))

1.86


In [8]:
print(np.mean(taxas_sucesso_chaves_geral))

0.414535


In [9]:
print(np.std(taxas_sucesso_chaves_geral))

0.11334772284876306
